In [1]:
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import csv
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

In [2]:
#Matplotlib 한국어 세팅
import platform
from matplotlib import font_manager, rc

plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname='C:\\Windows\\Fonts\\malgun.ttf').get_name()
    rc('font', family=font_name)

In [3]:
#palette
pal = ['#50d890', '#007944','#888888','#f3c623','#EFEFEF', '#96bb7c', '#d9bf77','#3f3f44']

In [15]:
card = pd.read_csv('data/card_20200717.csv')
region = pd.read_csv('data/adstrd_master.csv', encoding='UTF-8')
fpopl = pd.read_csv('data/fpopl.csv')
delivery = pd.read_csv('data/delivery.csv')

tour_num = pd.read_excel('data/외래객입국_목적별.xlsx',encoding='UTF-8') #한국관광 통계자료 검색(목적별)
tour_nation = pd.read_excel('data/외래객입국_목적별_국적별.xlsx',encoding='UTF-8') #한국관광 통계자료 검색(목적별_국적별)
tour_money = pd.read_excel('data/관광수지_관광수입.xlsx',encoding='UTF-8') #한국관광 통계자료 검색(관광수지_관광수입)
search_online = pd.read_excel('data/datalab.xlsx', encoding='UTF-8', header=6) #네이버 데이터랩 검색어 트렌드(e-commerce)
search_ott = pd.read_excel('data/datalab (1).xlsx', encoding='UTF-8', header=6) #네이버 데이터랩 검색어 트렌드(OTT)

In [16]:
card.head()

,receipt_dttm,adstrd_code,adstrd_nm,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt
0,20200104,1174066000,성내3동,7041,약국,463,5843230
1,20200104,1174066000,성내3동,7022,치과의원,33,7835550
2,20200104,1174066000,성내3동,7021,한의원,53,4589800
3,20200104,1174066000,성내3동,7020,의원,339,9267240
4,20200104,1174066000,성내3동,6110,자동차정비,19,4441000


### 전처리1. 날짜별, 업종별 합

In [17]:
card.columns = ['날짜', '행정동 코드', '행정동', '업종코드', '업종', '판매건수', '판매금액']
card['날짜'] = card['날짜'].astype('str').apply(lambda x: x[:4]+'-'+x[4:6]+'-'+x[6:])
card['행정동 코드'] = card['행정동 코드'].astype('str').apply(lambda x: x[:-2]).astype('int')

In [18]:
card['행정동 코드']

0          11740660
1          11740660
2          11740660
3          11740660
4          11740660
5          11740660
6          11740660
7          11740660
8          11740660
9          11740660
10         11740660
11         11740660
12         11740660
13         11740660
14         11740660
15         11740660
16         11740650
17         11740650
18         11740650
19         11740650
20         11740650
21         11740650
22         11740650
23         11740650
24         11740650
25         11740650
26         11740650
27         11740650
28         11740650
29         11740650
             ...   
3713922    11350595
3713923    11170685
3713924    11380552
3713925    11545700
3713926    11470510
3713927    11590520
3713928    11380625
3713929    11260610
3713930    11290685
3713931    11545710
3713932    11305635
3713933    11290650
3713934    11740540
3713935    11500560
3713936    11710590
3713937    11620605
3713938    11350619
3713939    11740520
3713940    11740520


In [19]:
region.head()

,adstrd_code,adstrd_nm,brtc_nm,signgu_nm
0,11110515.0,청운효자동,서울특별시,종로구
1,11110530.0,사직동,서울특별시,종로구
2,11110540.0,삼청동,서울특별시,종로구
3,11110550.0,부암동,서울특별시,종로구
4,11110560.0,평창동,서울특별시,종로구


In [20]:
region.columns = ['행정동 코드', '행정동', '광역시도', '시군도']
region['행정동 코드'] = region['행정동 코드'].astype('int')
card = pd.merge(card, region, on=['행정동 코드'], how='left')  # card와 region data를 '행정동 코드' 기준으로 합치기

card = card[card['광역시도']=='서울특별시']  # 서울특별시 data만 사용
card = card.drop(['행정동 코드', '업종코드'], axis=1)    # 
card = card.groupby(['날짜', '업종']).sum().reset_index()

In [21]:
card

,날짜,업종,판매건수,판매금액
0,2020-01-04,1급 호텔,38,6941145
1,2020-01-04,CATV홈쇼핑,4769,547207050
2,2020-01-04,가 방,83,5353600
3,2020-01-04,가전 제품,972,204308188
4,2020-01-04,가정용품수리,33,2047500
5,2020-01-04,건축요업품,63,36115540
6,2020-01-04,골프 용품,15,3052900
7,2020-01-04,골프연습장,34,2068625
8,2020-01-04,과학기자재,33,35589482
9,2020-01-04,관광여행,1063,852238104
